In [2]:
import librosa

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os,glob,pickle
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [4]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype = "float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result = np.hstack((result, mel))
    return result

In [5]:
def featureExtractor(audio):
    librosa_audio_data, librosa_sample_rate=librosa.load(audio)
    mfccs_features= librosa.feature.mfcc(y=librosa_audio_data, sr= librosa_sample_rate,n_mfcc=20)
    mfccs_scaled_features=np.mean(mfccs_features.T,axis=0)                             
    return mfccs_scaled_features

In [ ]:
# def load_data(test_size=0.2):
#     x,y = [],[]
#     for file in glob.glob(r"D:\BTP\infant cry\donateacry_corpus_cleaned_and_updated_data\*"):
#         label = os.path.basename(file)
#         for audio in glob.glob(file+"\*.wav"):
# #             print(audio)
#             feature = featureExtractor(audio)
#             x.append(feature)
#             y.append(label)
#     return train_test_split(np.array(x), np.array(y), test_size=test_size, random_state=101)

In [ ]:
# x_train, x_test, y_train, y_test=load_data()

In [ ]:
# print(len(x_train),len(y_train),len(x_test),len(y_test))
# print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

In [26]:
extracted_features=[]
def load_data():
    c=0
    for file in glob.glob(r"/content/drive/MyDrive/Baby_audio_dataset/*"):
        label = os.path.basename(file)
        for audio in glob.glob(file+"/*.wav"):
#             print(audio)
            if(label=='hungry'):
                if(c>200):
                    continue
                c+=1
            feature = featureExtractor(audio)
            extracted_features.append([feature,label])

In [27]:
load_data()

In [ ]:
# extracted_features

In [28]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-259.2603, 102.64332, -4.2214217, 9.362978, -...",Baby laugh
1,"[-274.79245, 118.66719, -10.167014, 3.5980012,...",Baby laugh
2,"[-417.6871, 106.74806, 4.0384803, -0.8012256, ...",Baby laugh
3,"[-262.63992, 101.5998, -4.972275, 9.254425, -2...",Baby laugh
4,"[-244.17845, 122.29594, -1.7869653, 3.554358, ...",Baby laugh


In [29]:
print(extracted_features_df.shape)

(452, 2)


In [ ]:
# audio_file_path='belly_pain.wav'
# mfcc=featureExtractor(audio_file_path)
# print(mfcc)
# print(mfcc.shape)

In [30]:
X=np.array(extracted_features_df['feature'].tolist())
Y=np.array(extracted_features_df['class'].tolist())

In [31]:
print(X.shape,Y.shape)
print(Y)

(452, 20) (452,)
['Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh'
 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh'
 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh'
 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh'
 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh'
 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh'
 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh'
 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh'
 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh'
 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh'
 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh'
 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh'
 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh'
 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh'
 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby laugh' 'Baby 

In [32]:
Y.shape

(452,)

In [33]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(Y))
# Y=np.array(pd.get_dummies(Y))

In [34]:
print(y.shape)
y

(452, 7)


array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [35]:
#train test split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [36]:
X_train.shape

(361, 20)

In [37]:
y_train.shape

(361, 7)

In [38]:
X_test.shape

(91, 20)

In [39]:
y_test.shape

(91, 7)

In [40]:
# BUILDING MODEL
import tensorflow as tf
print(tf.__version__)

2.8.2


In [41]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [42]:
num_labels=y.shape[1]

In [43]:
# Dense()

In [44]:
# ANN
model=Sequential()
#first layer
model.add(Dense(100,input_shape=(20,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
#second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
          
# final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [45]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               2100      
                                                                 
 activation (Activation)     (None, 100)               0         
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 200)               20200     
                                                                 
 activation_1 (Activation)   (None, 200)               0         
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 7)                 1

In [46]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [47]:
# Training
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

num_epochs=50
num_batch_size=32

checkpointer=ModelCheckpoint(filepath='saved_models/audio_classification.hdf5',
                             verbose=1,save_best_only=True)
start=datetime.now()
model.fit(X_train,y_train,batch_size=num_batch_size,epochs=num_epochs,
          validation_data=(X_test,y_test),callbacks=[checkpointer] )
duration=datetime.now() - start
print("Training completed in time: ",duration)

Epoch 1/50
 1/12 [=>............................] - ETA: 9s - loss: 62.6655 - accuracy: 0.0938
Epoch 1: val_loss improved from inf to 15.86618, saving model to saved_models/audio_classification.hdf5
12/12 [==============================] - 1s 25ms/step - loss: 42.4536 - accuracy: 0.2659 - val_loss: 15.8662 - val_accuracy: 0.4176
Epoch 2/50
 1/12 [=>............................] - ETA: 0s - loss: 20.1737 - accuracy: 0.4688
Epoch 2: val_loss improved from 15.86618 to 7.16429, saving model to saved_models/audio_classification.hdf5
12/12 [==============================] - 0s 9ms/step - loss: 27.9433 - accuracy: 0.3186 - val_loss: 7.1643 - val_accuracy: 0.5824
Epoch 3/50
 1/12 [=>............................] - ETA: 0s - loss: 25.1043 - accuracy: 0.3438
Epoch 3: val_loss improved from 7.16429 to 6.09286, saving model to saved_models/audio_classification.hdf5
12/12 [==============================] - 0s 9ms/step - loss: 21.4961 - accuracy: 0.3657 - val_loss: 6.0929 - val_accuracy: 0.5824
Epoc

In [48]:
loss,test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print("loss: ",loss)
print(test_accuracy)

loss:  0.7602384686470032
0.8021978139877319


In [56]:
filename='/content/drive/MyDrive/Baby_audio_dataset/Baby laugh/laugh_1.m4a_0.wav'
prediction_feature=featureExtractor(filename).reshape(1,-1)
# model.predict_classes(prediction_feature)
#predicted_label=model.predict_classes(prediction_feature)
predicted_label=model.predict(X_test) 
#classes_x=np.argmax(predicted_label,axis=1)
print(predicted_label)
#predcited_class=labelencoder.inverse_transform(predicted_label)
predicted_class=np.argmax(predicted_label,axis=1)
predicted_class
# (model.predict(prediction_feature) > 0.5).astype("int32")

[[1.18456759e-01 8.71693313e-01 1.20725273e-03 1.21163798e-03
  2.48265592e-03 1.73627667e-03 3.21204634e-03]
 [9.64695036e-01 2.97072232e-02 2.11346109e-04 9.62863123e-05
  2.68931151e-03 2.26365868e-03 3.37222882e-04]
 [1.26667041e-02 1.57748573e-02 8.61570165e-02 6.21259026e-02
  1.88863218e-01 5.13795972e-01 1.20616257e-01]
 [1.93046127e-02 2.00080518e-02 1.11394063e-01 1.36998594e-01
  1.40861481e-01 4.47646111e-01 1.23787068e-01]
 [4.14372236e-02 3.49204876e-02 1.37786463e-01 1.27234057e-01
  1.52530819e-01 3.20738703e-01 1.85352296e-01]
 [4.14668575e-05 9.99953389e-01 7.87085384e-08 8.27858969e-07
  8.85973577e-07 1.77464710e-06 1.60405386e-06]
 [6.45187795e-01 5.70006035e-02 7.65090762e-03 1.49684888e-03
  9.57313403e-02 1.00143030e-01 9.27893594e-02]
 [3.70117230e-03 4.77626640e-03 4.61574644e-02 7.20808655e-02
  1.31663695e-01 5.72267771e-01 1.69352680e-01]
 [1.72441781e-01 3.42681110e-02 5.25308028e-02 6.72403211e-03
  2.03391448e-01 3.74669462e-01 1.55974358e-01]
 [9.929884

array([1, 0, 5, 5, 5, 1, 0, 5, 5, 0, 5, 1, 5, 1, 5, 0, 1, 0, 0, 5, 5, 5,
       5, 1, 5, 5, 5, 4, 5, 5, 5, 5, 5, 0, 4, 5, 5, 5, 5, 5, 1, 5, 5, 5,
       5, 5, 0, 1, 2, 0, 5, 5, 1, 0, 0, 0, 4, 0, 0, 5, 1, 5, 5, 0, 4, 0,
       0, 5, 5, 5, 0, 5, 5, 1, 0, 5, 1, 5, 0, 1, 5, 5, 1, 5, 5, 1, 1, 5,
       5, 5, 0])

In [57]:
y_pred = model.predict(X_test) 
y_pred = np.argmax(y_pred, axis = 1) 
label = np.argmax(y_test,axis = 1) 

In [58]:
from sklearn.metrics import classification_report,confusion_matrix,f1_score
print(classification_report(y_true=label,y_pred=y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        21
           1       1.00      1.00      1.00        16
           2       0.00      0.00      0.00         3
           3       0.00      0.00      0.00         1
           4       0.25      0.17      0.20         6
           5       0.71      0.92      0.80        38
           6       0.00      0.00      0.00         6

    accuracy                           0.80        91
   macro avg       0.42      0.44      0.43        91
weighted avg       0.72      0.80      0.76        91



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [59]:
f1_score_per_class_validation = f1_score(y_true=label,y_pred=y_pred,average=None) 
print("The f1-score per class: ",f1_score_per_class_validation)
print("The f1-score : ",f1_score(y_true=label,y_pred=y_pred,average='weighted'))

The f1-score per class:  [1.        1.        0.        0.        0.2       0.8045977 0.       ]
The f1-score :  0.7557660730074522
